In [1]:
from utils import *
import os.path
from scipy.stats import qmc
from tqdm import tqdm
from pymoo.operators.sampling.lhs import LHS
import polars as pl

In [2]:
save_dir = 'scale_problem'
create_directory_if_not_exist(save_dir)

In [3]:
sample_size = 250 #times dimention
problem_dim = 5
seed=42
n_instance_reapets = 300

In [4]:
suite = list(generate_scaled_problems(problem_dim=problem_dim, seed=seed, n_runs=n_instance_reapets))

In [5]:
file = f'{save_dir}/samples__dim_{problem_dim}__samplesize_{sample_size}.parquet'

In [6]:
if os.path.isfile(file)==False:
    data = {'problem': [], 'instance': [], 'y':[], 'scale':[]}
    
    for problem in tqdm(suite):
        sampling = LHS()
        sampler = qmc.LatinHypercube(d=problem.n_var)
        sample = sampler.random(n=sample_size*problem.n_var)
        sample_scaled = qmc.scale(sample, problem.xl, problem.xu)
        data['problem'].extend(len(sample_scaled)*[problem.id_function])
        data['instance'].extend(len(sample_scaled)*[problem.id_instance])
        for r in range(problem.n_var):
            if f'x_{r}' not in data:
                data[f'x_{r}'] = []
            data[f'x_{r}'].extend(sample_scaled[:, r])
        ys = problem.evaluate(sample_scaled).flatten()
        data['y'].extend(ys)
        data['scale'].extend(len(ys)*[problem.scale])
    df = pl.DataFrame(data)
    df.write_parquet(file)
else:
    df = pl.read_parquet(file)

100%|██████████| 108000/108000 [07:34<00:00, 237.77it/s]


In [7]:
df

problem,instance,y,scale,x_0,x_1,x_2,x_3,x_4
i64,i64,f64,f64,f64,f64,f64,f64,f64
1,1,6.060719,0.041858,-4.296427,-3.104281,-0.192616,-0.673033,3.131601
1,1,6.126361,0.041858,4.528078,-3.067798,2.361273,0.416028,3.076782
1,1,3.877255,0.041858,-1.140813,-3.502411,-2.387861,1.560439,-1.006926
1,1,6.632123,0.041858,2.961459,2.547677,-0.053763,2.266459,4.891365
1,1,4.10911,0.041858,-2.809439,-0.757153,-0.830928,0.079946,-0.28717
…,…,…,…,…,…,…,…,…
24,15,7190.788579,29.199993,4.132589,1.248377,-1.640549,3.33526,-1.551975
24,15,8519.918924,29.199993,4.488035,2.981614,-3.87305,3.56724,-0.802508
24,15,8824.975638,29.199993,-3.955833,-1.963122,2.818195,-2.607644,-3.413899
